# English Wikipedia Animation using First and Second order LINE
Sometimes, expecially when preparing a presentation for a conference or your work colleagues, a good animation can say more than a thousand words.

For this reason, we have prepared a straighforward way to create animations for a number of tasks using GRAPE that, through subsampling, can be executed on graphs of arbitrary size.

In this brief tutorial, we will show how to get a Directed English Wikipedia embedding using Second-order LINE, and then we will use TSNE decoposition to reduce its dimensionality and plot it into a short video.

The resulting WEBM can be converted using one of many services and can be directly incorporated in Google Slides.

## Retrieving English Wikipedia
First, we retrieve English Wikipedia:

In [1]:
%%time
from grape.datasets.wikipedia import WikiEN

english_wikipedia = WikiEN(directed=True)

CPU times: user 2h 8min 49s, sys: 34.4 s, total: 2h 9min 24s
Wall time: 29min 52s


Then, let's take a look at its graph report:

In [2]:
english_wikipedia

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>WikiEN</h2><p class="paragraph">The directed multigraph WikiEN has 17.37M heterogeneous nodes and 140.81M heterogeneous edges. The RAM requirements for the nodes and edges data structures are 4.83GB and 425.23MB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 0, the maximum node degree is 11.56K, the mode degree is 0, the mean degree is 8.11 and the node degree median is 0.</p><p class="paragraph">The nodes with the highest degree centrality are index of singapore-related articles (degree 11.56K and node types internal_wiki, singapore-related lists and indexes of topics by country), alphabetical list of municipalities of italy (degree 7.96K and node types internal_wiki, lists of municipalities of italy and alphabetic lists), index of ancient greece-related articles (degree 6.81K and node types internal_wiki, ancient greece and ancient greece-related lists), iucn red list vulnerable species (animalia) (degree 6.01K and node types internal_wiki and iucn red list vulnerable species) and index of telangana-related articles (degree 5.51K and node types internal_wiki, indexes of topics by country and telangana-related lists).</p><h3 style="margin: 1em 0 0 0;">Node types</h3><p class="paragraph">The graph has 1.47M node types, of which the 10 most common are websites (6.53M nodes, 37.60%), internal_wiki (6.29M nodes, 36.23%), external_wiki (4.55M nodes, 26.17%), living people (1.02M nodes, 5.88%), year of birth missing (living people) (143.06K nodes, 0.82%), association football midfielders (59.53K nodes, 0.34%), place of birth missing (living people) (57.68K nodes, 0.33%), american films (56.71K nodes, 0.33%), association football defenders (46.20K nodes, 0.27%) and association football forwards (44.44K nodes, 0.26%). The node types are multi-label, and the node with most node types has 246 node types. The RAM requirement for the node types data structure is 1.29GB.</p><h4 style="margin: 1em 0 0 0;">Approximated isomorphic node types</h4><p class="paragraph">Approximated isomorphic node types groups are node types describing exactly the same set of nodes. Since the graph has a high number of node types (> 50.00K) we use an approximated version of the node types isomorphisms detection. Note that hash collisions and false positives are possible. It is possible to execute the exact computation with the <code>get_isomorphic_node_type_ids_groups</code> method. The presence of such duplicated node types suggests a potential modelling error in the pipeline that has produced this graph. 38.08K isomorphic node types groups were detected in this graph.</p><ol><li><p class="paragraph">Isomorphic node type group containing 2 node types (161 nodes), which are: {{title year range}} nhl season by team and {{title year range}} in american ice hockey by team.</p></li>
<li><p class="paragraph">Isomorphic node type group containing 2 node types (116 nodes), which are: songs written by adam clayton and songs written by 

## Connected holdout
Since we want to visualize an edge prediction task on this graph, we need to create a connected holdout:

In [4]:
%%time
train, test = english_wikipedia.connected_holdout(train_size=0.7)
train.enable()

CPU times: user 2h 11min 48s, sys: 4.64 s, total: 2h 11min 53s
Wall time: 8min 17s


## Compute the embedding
Next, we compute the embedding using the Second-order LINE method. Do note that this implementation is a data-race aware one that uses SGD as optimizer, and nothing fancy like ADAM or NADAM: this means that the memory footprint is only limited to the embedding size.

In [5]:
from grape.embedders import SecondOrderLINEEnsmallen

In [6]:
%%time
second_order_embedding = SecondOrderLINEEnsmallen().fit_transform(train)

/home/luca/anaconda3/lib/python3.9/site-packages/embiggen/utils/abstract_models/abstract_embedding_model.py:166: UserWarning: Please be advised that the WikiEN train graph contains 21562 disconnected nodes. Consider that node embedding algorithms that only use topological information such as CBOW, GloVe, SPINE and SkipGram are not able to provide meaningful embeddings for these nodes, and their embedding will be generally far away from any other node. It is also possible that all disconnected nodes will receive a relatively similar node embedding. Consider dropping them by using the `graph.remove_disconnected_nodes()` method.
  warnings.warn(


CPU times: user 5h 46min 54s, sys: 50.6 s, total: 5h 47min 45s
Wall time: 15min 16s


## Visualize the embedding on the test graph
We are at the end, finally visualizing the test graph.

In [8]:
%%time
from grape import GraphVisualizer

vis = GraphVisualizer(
    graph=test,
    support=train,
    n_components=4,
    edge_embedding_method="Hadamard",
    rotate=True,
    verbose=True,
    # Automatically, since LINE learns a cosine, the visualization tool
    # would dispatch a Cosine-distance based TSNE. This would use the sklearn
    # implementation, which is terribly slow. Therefore, we force it to use the Euclidean distance
    # and therefore the Multicore TSNE implementation (when available).
    decomposition_kwargs=dict(metric="euclidean")
)

Then we run the TSNE, this may take a while.

In [9]:
%%time
vis.fit_negative_and_positive_edges(second_order_embedding)

Performing t-SNE using 24 cores.
Using no_dims = 4, perplexity = 30.000000, and theta = 0.500000
Computing input similarities...
Building tree...
 - point 2000 of 20000
 - point 4000 of 20000
 - point 6000 of 20000
 - point 8000 of 20000
 - point 10000 of 20000
 - point 12000 of 20000
 - point 14000 of 20000
 - point 16000 of 20000
 - point 18000 of 20000
 - point 20000 of 20000
Done in 1.00 seconds (sparsity = 0.007598)!
Learning embedding...
Iteration 51: error is 102.865125 (50 iterations in 5.00 seconds)
Iteration 101: error is 102.725834 (50 iterations in 7.00 seconds)
Iteration 151: error is 102.540174 (50 iterations in 8.00 seconds)
Iteration 201: error is 102.513155 (50 iterations in 8.00 seconds)
Iteration 251: error is 102.511385 (50 iterations in 8.00 seconds)
Iteration 301: error is 4.877847 (50 iterations in 5.00 seconds)
Iteration 351: error is 4.600061 (50 iterations in 6.00 seconds)
Iteration 400: error is 4.445273 (50 iterations in 5.00 seconds)


CPU times: user 7min 47s, sys: 13min 20s, total: 21min 8s
Wall time: 57.4 s


Fitting performed in 52.00 seconds.


In [10]:
%%time
vis.plot_positive_and_negative_edges()

Rendering frames:   0%|                                                                                       …

OpenCV: FFMPEG: tag 0x30387076/'vp80' is not supported with codec id 139 and format 'webm / WebM'


Merging frames:   0%|                                                                                         …

CPU times: user 9min 35s, sys: 12.2 s, total: 9min 47s
Wall time: 1min 36s
